# Working with Edgar datasets: Wrangling, Pre-processing and exploratory data analysis

In [1]:
#!pip install requests

In [2]:
#!pip install pandas

In [3]:
#!pip install lxml

In [14]:
#!pip install beautifulsoup4

In [71]:
from lxml import html
from lxml.html import parse
import requests
import html5lib
import pandas as pd
from pandas.io.parsers import TextParser
from urllib.request import urlopen
from bs4 import BeautifulSoup
import csv
import sys
import zipfile

Input variables (need to set as environmental variables for Docker pipline)

In [48]:
cik = '320193'
acc_no = '0000320193-17-000009'
acc_short = acc_no[0:10] + acc_no[11:13] + acc_no[14:]
print(acc_short)   

000032019317000009


Generate full URL and get the page

In [19]:
page = requests.get('https://www.sec.gov/Archives/edgar/data/' + cik + '/' + acc_short + '/' + acc_no +'-index.htm')
source = page.content
#print(source)

Scrape page content for links, find 10Q

In [20]:
htmlcontent = html.document_fromstring(source)
#print(htmlcontent)

In [49]:
#NEW

page = 'https://www.sec.gov/Archives/edgar/data/' + cik + '/' + acc_short + '/' + acc_no +'-index.htm'
soup1 = BeautifulSoup(requests.get(page).text, 'lxml')


In [5]:
#NEW

tables=soup1.findAll('table')
#print(tables[1])
table_count=0

In [7]:
#NEW

print(tables[0])

<table class="tableFile" summary="Document Format Files">
<tr>
<th scope="col" style="width: 5%;"><acronym title="Sequence Number">Seq</acronym></th>
<th scope="col" style="width: 40%;">Description</th>
<th scope="col" style="width: 20%;">Document</th>
<th scope="col" style="width: 10%;">Type</th>
<th scope="col">Size</th>
</tr>
<tr>
<td scope="row">1</td>
<td scope="row">10-Q</td>
<td scope="row"><a href="/Archives/edgar/data/51143/000005114313000007/ibm13q3_10q.htm">ibm13q3_10q.htm</a></td>
<td scope="row">10-Q</td>
<td scope="row">7305482</td>
</tr>
<tr class="blueRow">
<td scope="row">2</td>
<td scope="row">EX-11</td>
<td scope="row"><a href="/Archives/edgar/data/51143/000005114313000007/ibm13q3_ex11.htm">ibm13q3_ex11.htm</a></td>
<td scope="row">EX-11</td>
<td scope="row">81348</td>
</tr>
<tr>
<td scope="row">3</td>
<td scope="row">EX-12</td>
<td scope="row"><a href="/Archives/edgar/data/51143/000005114313000007/ibm13q3_ex12.htm">ibm13q3_ex12.htm</a></td>
<td scope="row">EX-12</td

In [8]:
rows = tables[0].findAll('tr')
clean_table = [] 
    
for row in rows:
    clean_row = []
    columns = row.find_all('td')
    for column in columns:
        clean_row.append(column.get_text().replace("\n","").replace("\xa0",""))
    clean_table.append(clean_row)
    #print(clean_row)
frame = pd.DataFrame(clean_table)
print(frame)

      0                              1                         2        3  \
0  None                           None                      None     None   
1     1                           10-Q           ibm13q3_10q.htm     10-Q   
2     2                          EX-11          ibm13q3_ex11.htm    EX-11   
3     3                          EX-12          ibm13q3_ex12.htm    EX-12   
4     4                        EX-31.1        ibm13q3_ex31d1.htm  EX-31.1   
5     5                        EX-31.2        ibm13q3_ex31d2.htm  EX-31.2   
6     6                        EX-32.1        ibm13q3_ex32d1.htm  EX-32.1   
7     7                        EX-32.2        ibm13q3_ex32d2.htm  EX-32.2   
8        Complete submission text file  0000051143-13-000007.txt            

          4  
0      None  
1   7305482  
2     81348  
3     35495  
4     10428  
5     11028  
6      6773  
7      7623  
8  26894261  


In [50]:
dfs = pd.read_html(page, header=0)

In [51]:
documents = dfs[0]

In [52]:
row10q= documents.loc[documents['Type'] == '10-Q']

In [53]:
print(row10q['Document'])

0    a10-qq32017712017.htm
Name: Document, dtype: object


In [54]:
url10q='https://www.sec.gov/Archives/edgar/data/' + cik + '/' + acc_short + '/' + str(row10q['Document'][0])


In [21]:
href = ''
for elem in htmlcontent.iterlinks():
    #print(elem[2])
    if "10q.htm" in elem[2]:
        href+= elem[2]

In [55]:
print(url10q)

https://www.sec.gov/Archives/edgar/data/320193/000032019317000009/a10-qq32017712017.htm


In [22]:
url10q = 'https://www.sec.gov' + href
print(url10q)

https://www.sec.gov/Archives/edgar/data/51143/000005114313000007/ibm13q3_10q.htm


In [69]:
test = pd.read_html(url10q)

### Scrape Tables with BeautifulSoup

In [66]:
test[3]

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Three Months Ended,NaN,Nine Months Ended,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,"July 1, 2017",NaN,"June 25, 2016",NaN,"July 1, 2017",NaN,"June 25, 2016",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Net sales,$,45408.00,NaN,NaN,$,42358.00,NaN,NaN,$,176655.00,NaN,NaN,$,168787.00,NaN
4,Cost of sales,27920,NaN,NaN,26252.0,NaN,NaN,108400,NaN,NaN,102337.00,NaN,NaN,NaN,NaN,NaN
5,Gross margin,17488,NaN,NaN,16106.0,NaN,NaN,68255,NaN,NaN,66450.00,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Operating expenses:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Research and development,2937,NaN,NaN,2560.0,NaN,NaN,8584,NaN,NaN,7475.00,NaN,NaN,NaN,NaN,NaN
9,"Selling, general and administrative",3783,NaN,NaN,3441.0,NaN,NaN,11447,NaN,NaN,10712.00,NaN,NaN,NaN,NaN,NaN


In [45]:
soup = BeautifulSoup(requests.get(url10q).text, 'lxml')
csvout  = csv.writer(sys.stdout)

In [46]:
tables=soup.findAll('table')
#print(tables[1])
table_count=0

In [47]:
for table in tables:
    rows = table.findAll('tr')
    clean_table = [] 
    
    for row in rows:
        clean_row = []
        columns = row.find_all('td')
        for column in columns:
            clean_row.append(column.get_text().replace("\n","").replace("\xa0",""))
        clean_table.append(clean_row)
        #print(clean_row)
    frame = pd.DataFrame(clean_table)
    #print(frame)
    csv_name = acc_no + "_" + str(table_count) + ".csv"
    frame.to_csv(csv_name)
    table_count +=1

In [73]:
table_count = 0
zipname = "10QTables_" + acc_no + ".zip"
z = zipfile.ZipFile(zipname, "w")

for table in tables:
    frame = pd.DataFrame(table)
    #logger.info('cleaning table...')
    csv_name = acc_no + "_" + str(table_count) + ".csv"
    frame.to_csv(csv_name)
    # logger.info(csv_name + ' created')
    z.write(csv_name)
    #logger.info(csv_name + ' added to zip file')
    table_count +=1

TypeError: DataFrame constructor called with incompatible data and dtype: setting an array element with a sequence